In [64]:
import pandas as pd

pd.set_option("display.max_rows", 300)
import rapidfuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import awswrangler as wr
import boto3
import copy
import itertools
from pprint import pprint


In [65]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
srcBucketName = "multilabel_df_3"
labelDf = wr.s3.read_parquet(
    path=f"s3://{srcBucketName}/",
    dataset=True
)
labelDf

,vision_bonnet,vision_bumper_front,vision_engine,vision_grille,vision_headlamp_lh,vision_headlamp_rh,vision_door_front_lh,vision_door_front_rh,vision_bumper_rear,vision_misc,...,Assembly_Type,Vehicle_Still_Driveable,NCB_Stat,Claim_Type,Vehicle_Type,Sum_Insured,Repairer,Repairer_Apprv_Count,Collision_With,Handling_Insurer
0,0,1,1,0,0,0,0,0,0,0,...,3,1,1,OD,Hatchback - 5 Dr,39000.0,Marathon Motor Sdn Bhd (Marathon),1405,Private Vehicle,Zurich General Takaful Malaysia Berhad
1,0,0,0,0,0,0,0,0,1,0,...,1,1,0,OD KFK,MPV,0.0,Syarikat Bengkel Soan Huat Sdn Bhd (HQ),990,Private Vehicle,Takaful Ikhlas General Berhad
2,1,1,1,1,1,1,0,0,0,1,...,3,0,1,OD,Saloon - 4 Dr,43000.0,Ichiban Chiba Auto Service Sdn Bhd (HQ),954,Others,MSIG INSURANCE (MALAYSIA) BHD
3,0,1,1,0,1,0,0,0,0,1,...,3,0,1,OD,Pickup - 4 Dr Dbl.Cab,125000.0,Hiewa Auto Gallery (Melaka) Sdn Bhd (Hiewa Mel...,2343,Private Vehicle,Etiqa General Insurance Berhad
4,0,0,0,0,0,0,0,0,0,1,...,3,1,0,OD KFK,Saloon - 4 Dr,44000.0,Peringgit Sri Auto Services Sdn Bhd (HQ Puchong),12375,Private Vehicle,Tokio Marine Insurans (Malaysia) Berhad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583810,0,1,1,1,0,0,0,0,0,1,...,3,0,1,OD,SUV - 5 Dr,103000.0,Hoi Keen Service Sdn Bhd (Hoi Keen),3235,Private Vehicle,Kurnia Insurans (AmGeneral Insurance Berhad)
583811,0,0,0,0,0,0,0,0,1,0,...,3,1,0,OD KFK,Saloon - 4 Dr,85000.0,GMV Motors Sdn Bhd (HQ),692,Private Vehicle,The Pacific Insurance Bhd
583812,0,1,0,0,0,0,0,0,0,0,...,3,1,1,OD,Saloon - 4 Dr,35000.0,Perodua Sales Sdn Bhd (B & P Hub Penang),4852,Animal,Zurich General Takaful Malaysia Berhad
583813,0,0,0,0,0,0,0,0,1,0,...,3,0,1,OD,MPV,20600.0,Syarikat Motor Sin Wing Soon (Klg) Sdn Bhd (HQ),7098,Private Vehicle,Allianz General Insurance Company (Malaysia) B...


In [66]:
labelDf[labelDf["CaseID"] == 10004193]

,vision_bonnet,vision_bumper_front,vision_engine,vision_grille,vision_headlamp_lh,vision_headlamp_rh,vision_door_front_lh,vision_door_front_rh,vision_bumper_rear,vision_misc,...,Assembly_Type,Vehicle_Still_Driveable,NCB_Stat,Claim_Type,Vehicle_Type,Sum_Insured,Repairer,Repairer_Apprv_Count,Collision_With,Handling_Insurer


In [67]:
labelDf.columns

Index(['vision_bonnet', 'vision_bumper_front', 'vision_engine',
       'vision_grille', 'vision_headlamp_lh', 'vision_headlamp_rh',
       'vision_door_front_lh', 'vision_door_front_rh', 'vision_bumper_rear',
       'vision_misc', 'vision_front_panel', 'vision_non_external',
       'vision_wheel', 'vision_fender_front_lh', 'vision_fender_front_rh',
       'vision_rear_quarter_lh', 'vision_tail_lamp_lh', 'vision_tail_lamp_rh',
       'vision_windscreen_front', 'vision_rear_compartment',
       'vision_rear_panel', 'vision_rear_quarter_rh', 'vision_door_rear_rh',
       'vision_door_rear_lh', 'vision_windscreen_rear', 'bonnet',
       'bumper_front', 'air_conditioning', 'cooling_fan', 'grille',
       'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh',
       'radiator', 'bumper_rear', 'misc', 'engine', 'airbag', 'front_panel',
       'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh',
       'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh',
       

In [68]:
mappingDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/tmp/complete_view_mapping.csv")

In [69]:
mappingDf["lvl_3_desc"] = mappingDf["lvl_3_desc"].str.strip()

In [70]:
view = ["front_view", "rear_view", "front_view_left", "front_view_right", "rear_view_left", "rear_view_right"]
viewToPart = dict()
for v in view:
    partsInView = mappingDf[mappingDf[v] == 1]["lvl_3_desc"].unique().tolist()
    viewToPart[v] = ["vision_" + x.replace(" ", "_") for x in partsInView]
pprint(viewToPart)
    

{'front_view': ['vision_bonnet',
                'vision_bumper_front',
                'vision_engine',
                'vision_front_panel',
                'vision_grille',
                'vision_headlamp_lh',
                'vision_headlamp_rh',
                'vision_misc',
                'vision_non_external',
                'vision_windscreen_front'],
 'front_view_left': ['vision_bonnet',
                     'vision_bumper_front',
                     'vision_door_front_lh',
                     'vision_door_rear_lh',
                     'vision_engine',
                     'vision_fender_front_lh',
                     'vision_front_panel',
                     'vision_grille',
                     'vision_headlamp_lh',
                     'vision_non_external',
                     'vision_misc',
                     'vision_wheel',
                     'vision_windscreen_front'],
 'front_view_right': ['vision_bonnet',
                      'vision_bumper_front',
    

In [71]:
vehicleType= "SUV - 5 Dr"
localFileMetadata = f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/imgs_metadata/{vehicleType}.parquet"

In [72]:
localFilesDf = pd.read_parquet(localFileMetadata)
localFilesDf["StdDocDesc"] = localFilesDf["StdDocDesc"].str.replace(" ", "_").str.lower()
localFilesDf.sort_values(by="CaseID", inplace=True)
localFilesDf

,CaseID,iDOCID,StdDocDesc,filename,url
1692766,10000064,440265957.0,rear_view_left,10000064_440265957.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
2290887,10000064,440265956.0,rear_view_right,10000064_440265956.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
286191,10000064,440265954.0,front_view_left,10000064_440265954.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
884312,10000064,440265955.0,front_view_right,10000064_440265955.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
4530,10000064,440567975.0,front_view,10000064_440567975.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
...,...,...,...,...,...
1470462,13562831,619818878.0,front_view_right,13562831_619818878.JPG,http://192.168.1.4:8888/buckets/raw_imgs/13562...
2278894,13562831,619818888.0,rear_view_left,13562831_619818888.JPG,http://192.168.1.4:8888/buckets/raw_imgs/13562...
2877014,13562831,619818883.0,rear_view_right,13562831_619818883.JPG,http://192.168.1.4:8888/buckets/raw_imgs/13562...
872330,13562831,619818874.0,front_view_left,13562831_619818874.JPG,http://192.168.1.4:8888/buckets/raw_imgs/13562...


In [73]:
viewDfMap = dict()
for v, parts in viewToPart.items():
    tempDf = labelDf[parts + ["CaseID"]]
    tempDf["view"] = v
    viewFilesDf = localFilesDf[localFilesDf["StdDocDesc"] == v]
    viewFilesDf.drop_duplicates(subset=["CaseID"], inplace=True)
    tempDf = tempDf.merge(viewFilesDf[["filename", "CaseID"]], on="CaseID")
    viewDfMap[v] = tempDf


/tmp/ipykernel_26515/903975900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDf["view"] = v
/tmp/ipykernel_26515/903975900.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  viewFilesDf.drop_duplicates(subset=["CaseID"], inplace=True)
/tmp/ipykernel_26515/903975900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [74]:
for view, df in viewDfMap.items():
    imgLabelFilename = f"{vehicleType}_{view}_img_labels.csv"
    wr.s3.to_csv(
        df=df,
    path=f"s3://imgs_labels_4/{imgLabelFilename}",
    

    )

In [75]:
for view, df in viewDfMap.items():
    imgLabelFilename = f"{vehicleType}_{view}_img_labels.csv"
    df = wr.s3.read_csv(
    path=f"s3://imgs_labels_4/{imgLabelFilename}",

    )
    print(df)

      Unnamed: 0  vision_bonnet  vision_bumper_front  vision_engine  \
0              0              0                    1              0   
1              1              0                    0              1   
2              2              0                    1              1   
3              3              0                    0              0   
4              4              1                    1              1   
...          ...            ...                  ...            ...   
3656        3656              0                    1              0   
3657        3657              1                    1              1   
3658        3658              1                    1              1   
3659        3659              0                    1              0   
3660        3660              1                    1              1   

      vision_front_panel  vision_grille  vision_headlamp_lh  \
0                      0              1                   0   
1                    

: 